In [ ]:
!pip install gym_super_mario_bros

In [12]:
import cv2
import gym
import torch
import math
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [14]:
import torch.nn as nn
import torch.optim as optim

# define the CNN_RNN model
class CNN_RNN(nn.Module):
    def __init__(self):
        super(CNN_RNN, self).__init__()
        self.conv1 = nn.Conv2d(4, 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.rnn = nn.LSTM(7*7*64, 512, batch_first=True)
        self.fc = nn.Linear(512, 5)

    def forward(self, x, hx=None):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.relu(self.conv3(x))
        x = x.view(-1, 7*7*64)
        if hx is not None:
            hx = (hx[0].contiguous(), hx[1].contiguous())
            x, hx = self.rnn(x.unsqueeze(0), hx)
            x = self.fc(x.squeeze(0))
            return x, hx
        else:
            x, _ = self.rnn(x.unsqueeze(0))
            x = self.fc(x.squeeze(0))
            return x

In [24]:
# defining the training parameters
learning_rate = 0.0001
gamma = 0.99
num_episodes = 10
batch_size = 32
update_frequency = 4
target_update_frequency = 1000

In [25]:
# creating the environment
env = gym.make('SuperMarioBros-v0')

# create the CNN_RNN model and optimizer
model = CNN_RNN()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [26]:
# defining a loss function
def compute_loss(state, action, reward, next_state, done):
    # calculation of the Q-values for the current state and action
    q_values, _ = model(state)
    q_value = q_values.gather(1, action.unsqueeze(1))

    # calculatation of the Q-values for the next state
    next_q_values, _ = model(next_state)
    next_q_value, _ = torch.max(next_q_values, 1, keepdim=True)

    # calculation of the target Q-value
    target_q_value = reward + gamma * next_q_value * (1 - done)

    # MSE loss
    loss = nn.functional.mse_loss(q_value, target_q_value)

    return loss

In [27]:
# defining the epsilon-greedy exploration policy
epsilon_start = 1.0
epsilon_end = 0.1
epsilon_decay = 1000000
epsilon_by_frame = lambda frame_idx: epsilon_end + (epsilon_start - epsilon_end) * \
    math.exp(-1. * frame_idx / epsilon_decay)

### Coding Replay Buffer Class

In [28]:
import random
from collections import deque
import numpy as np

class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))
    
    def sample(self, batch_size):
        states, actions, rewards, next_states, dones = zip(*random.sample(self.buffer, batch_size))
        return np.array(states), np.array(actions), np.array(rewards), np.array(next_states), np.array(dones)
    
    def __len__(self):
        return len(self.buffer)

In [29]:
# define the replay buffer
replay_buffer = ReplayBuffer(10000)

# define the target network
target_model = CNN_RNN()
target_model.load_state_dict(model.state_dict())
target_model.eval()

CNN_RNN(
  (conv1): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
  (conv2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (rnn): LSTM(3136, 512, batch_first=True)
  (fc): Linear(in_features=512, out_features=5, bias=True)
)

In [30]:
# defining the frame counter
frame_idx = 0

In [ ]:
for episode in range(num_episodes):
    state = env.reset()
    episode_reward = 0
    done = False
    hx = None

    while not done:
        # choosing an action using the epsilon-greedy policy
        if random.random() < epsilon_by_frame(frame_idx):
            action = env.action_space.sample()
        else:
          state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
          action_tensor = torch.tensor([action], dtype=torch.long)
          next_state, reward, done, _ = env.step(action)
          replay_buffer.push(state, action, reward, next_state, done)
          state = next_state
          episode_reward += reward
          frame_idx += 1

           # sample a batch of transitions from the replay buffer
        if len(replay_buffer) > batch_size:
            state_batch, action_batch, reward_batch, next_state_batch, done_batch = \
                replay_buffer.sample(batch_size)

            # calculate the loss and update the model
            loss = compute_loss(
                torch.tensor(state_batch, dtype=torch.float32),
                torch.tensor(action_batch, dtype=torch.long),
                torch.tensor(reward_batch, dtype=torch.float32),
                torch.tensor(next_state_batch, dtype=torch.float32),
                torch.tensor(done_batch, dtype=torch.float32)
            )
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # update the target network
            if frame_idx % target_update_frequency == 0:
                target_model.load_state_dict(model.state_dict())
    # print the episode results
    print(f"Episode {episode}: reward = {episode_reward}")

    # save the model
    if (episode + 1) % 100 == 0:
        torch.save(model.state_dict(), f"model_episode_{episode+1}.pt")